# This notebook present the most basic use of Grid2Op

**Objectives**

This notebook will cover some basic raw functionality at first. It will then show how these raw functionalities are encapsulated with easy to use functions.

The recommended way to use these is to through the Runner, and not by getting through the instanciation of class one by one.

In [1]:
import os
import sys
import grid2op

In [2]:
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

## 0) Summary of RL method

Though the `Grid2Op` package can be used to perform many different tasks, these set of notebooks will be focused on the machine learning part, and its usage in a Reinforcement learning framework. 

The reinforcement learning is a framework that allows to train "agent" to solve time dependant domain. We tried to cast the grid operation planning into this framework. The package `Grid2Op` was inspired by it.

In a reinforcement learning (RL), there are 2 distinct entities:
* **Environment**: is a modeling of the "world" in which the *agent* takes some *actions* to achieve some pre definite objectives.
* **Agent**: will do actions on the environment that will have consequences.

These 2 entities exchange 3 main type of information:
* **Action**: it's an information sent by the Agent that will modify the internal state of the environment.
* **State** / **Observation**: is the (partial) view of the environment by the Agent. The Agent receive a new state after each actions. He can use the observation (state) at time step *t* to take the action at time *t*.
* **Reward**: is the score received by the agent for the previous action.

A schematic representaiton of this is shown in the figure bellow (Credit: [Sutton & Barto](http://incompleteideas.net/book/bookdraft2017nov5.pdf)):

![title](img/reinforcement-learning.jpg)

In this notebook, we will develop a simple Agent that takes some action (powerline disconnection) based on the observation of the environment.

For more information about the problem, please visit the [Example_5bus](Example_5bus.ipynb) notebook which dive more into the casting of the real time operation planning into a RL framework. Note that this notebook is still under development at the moment.

A good material is also provided in the white paper [Reinforcement Learning for Electricity Network Operation](https://arxiv.org/abs/2003.07339) presented for the L2RPN 2020 Neurips edition.

## I) Creating an Environement: Step by Step explanation of the basic classes of this package

### I.A) Get Data to feed the powergrid

In order to be initialized, an Agent need to know in which space he operates. For that, we load an Environement, based on the IEEE case14.

An example of this powergrid can be found in the package data. We import them here:

In [3]:
powergrid_path = grid2op.CASE_14_FILE
multi_episode_path = grid2op.CHRONICS_MLUTIEPISODE
max_iter = 10

***NB*** In order to work smoothly, the backend and the data in the files need to have the exact same name. As often the process to generate the data (we suppose that the equilibrium between productions and load is performed beforehand) is different and agnostic from the powergrid, it is not surpising to have the same physical object with different name in the data for the temporal series and for the powergrid file description. In order to simplify the matching, in this example we use the mapper `names_chronics_to_backend` that is able to "convert" the names given in the data to the names of the objects in the powergrid description file. It's a mapper that makes the link between the two.

More detail about how the matching is performed can be found in the help of the *ChronicsHandler.GridValue.initialize* method [here](https://grid2op.readthedocs.io/en/latest/chronics.html#grid2op.Chronics.GridValue.initialize) or in the file [ChronicsHandler.py](../grid2op/Chronics/ChronicsHandler.py)

In order to work, an *Environement* need to be fed with data. These data can be read from files for example. Some examples are also provided in this package. We import them:

In [4]:
from grid2op.Chronics import ChronicsHandler, Multifolder, GridStateFromFileWithForecasts
data_feeding = ChronicsHandler(chronicsClass=Multifolder,
                               path=multi_episode_path,
                               gridvalueClass=GridStateFromFileWithForecasts,
                               max_iter=max_iter)

`data_feeding` is now an instance that will automatically load the data and modify the powergrid accordingly at each time step. The process of reading is handled by this class, but the process of modifying the underlying powergrid is carried out by the *Environment* and performed by the *Backend*.

### I.B) Get a Backend to carry out the computations

A *Backend* is a dedicated object that has the responsability to compute the resulting powerflow from given injections (poductions and loads). The possibility to implement your own Backend make the Grid2Op framework completely agnostic of the modeling of the powergrid you want to use, or the method to solve the powerflow.

A implementation of a Backend is provided with Pandapower.

In [5]:
from grid2op.Backend import PandaPowerBackend
backend = PandaPowerBackend()

`backend` is now a variable that is able to compute powerflow and is able to emulate cascading failures. To be able to work properly and carry out the right computations, the worker need to be aware of some *Parameters*.

### I.C) Getting the parameters of the game

For this example, we will use the default parameters available. More information about the parameters that can be modified can be found in the help [here](https://grid2op.readthedocs.io/en/latest/parameters.html), or in the file [Parameters.py](grid2op/Parameters.py).

In [6]:
from grid2op.Parameters import Parameters
param = Parameters()

**Note on the parameters**: some parameters here have direct influence on the difficulty of the game. For example: "NO_OVERFLOW_DISCONNECTION" member of this class, if set to ``True`` (default is ``False``) will not trigger the automatic disconnection of powerline when they are in overflow. While some other parameters can have a influence on the speed with which some timesteps will be computed. This is for example the case with the member "ENV_DC". If this member "ENV_DC" is set to ``True`` a faster (but less accurate) computation engine will be used to compute the flows. This might be usefull for example at the beginning of training.

To set the paramters, you can do, for example:

```python
from grid2op.Parameters import Parameters
param = Parameters()
param.from_dict({"NO_OVERFLOW_DISCONNECTION": True, "ENV_DC": True})
```

### I.D) Building the Environment

In [7]:
from grid2op.Environment import Environment
env = Environment(init_grid_path=powergrid_path,
                  chronics_handler=data_feeding,
                  backend=backend,
                  parameters=param)

Creating an *Environment* will load the powergrid (*powergrid_path*), load the data to feed it (*data_feeding*), network powerflow simulator (*backend*) and game settings (*param*. Use the first row to initialize the powergrid and some other internal checking that the data are suited to the powergrid.

It can take a moment (usually a few seconds) to load it.

This environment can be greatly customize. We expose here only basic functionnalities. For more information it is advised to read the documentation [here](https://grid2op.readthedocs.io/en/latest/environment.html) (if it has been built locally), to consult the official documentation on the internet or to consult the source code of [Environment.py](grid2op/Environment/Environment.py)

### I.E) A shortcut to perfom all that

All of the above can be done in calling one function that will handle the creation of the Environment with default values.

This particular above mentioned environment is named `rte_case14_redisp`. As its name said it is a testing environment

To define/create it, we can call:

In [8]:
env = grid2op.make_new("rte_case14_redisp", local=True)

I used testing dataset


/home/donnotben/Documents/Grid2Op_dev/getting_started/grid2op/MakeEnv/MakeNew.py:140: UserWarning:

You are using a development environment. This is really not recommended for training agents.



**NB** By setting "local=True" in the above call, we use only 2 different months or for our environment. If you remove it, grid2op.make_new will attempt to download more data. By default data corresponding to this environment will be downloaded in your "home" directory, which correspond to the location returned by this script:

```python
import os
print(f"grid2op dataset will be downloaded in {os.path.expanduser('~/data_grid2op')}")
```

If you want another default saving path, you can add a .grid2opconfig.json and specify where to download the data.

Only 4 environment are available locally when you install grid2op:
- `rte_case5_example`, only available locally, its main goal is to provide example on really small system that people can really study manually and have intuitions on it.
- `rte_case14_test`, only available locally, its main goal is to be used inside unit test of grid2op. It is **NOT** recommended to use it.
- `rte_case14_redisp` an environment based on the IEEE case14 files, which introduces the redispatching. Only 2 dataset are available for this environment. More can be downloaded automatically by specifying "local=False" [default value of argument "local"]
- `rte_case14_realistic` an environment "realistic" based on the same grid which has been adapted to be closer to "real" grid. More can be downloaded automatically by specifying "local=False" [default value of argument "local"]

Other environments can be used and are available through the "make_new" command, to get a list of the possible environments you can do:

In [9]:
grid2op.list_available_remove_env()

['rte_case14_realistic', 'rte_case14_redisp']

**NB** In future release, make_new will become the new "make" and make will be moved to "make_old". Then "make_old", which is the current "make" will disapear. We encourage you to update your scripts accordingly.

## II) Creating an Agent

An *Agent* is the name given to the "operator" / "bot" / "algorithm" that will perform some modification of the powergrid when he faces some "observation".

Some example of Agents are provided in the file [Agent.py](grid2op/Agent/Agent.py).

A deeper look at the different Agent provided can be found in the [4_StudyYourAgent](4_StudyYourAgent.ipynb.ipynb) notebook (in progress). We suppose here we use the most simple Agent, the one that does nothing

In [10]:
from grid2op.Agent import DoNothingAgent
my_agent = DoNothingAgent(env.helper_action_player)

## III) Assess how the Agent is performing

The performance of each Agent is assessed with the reward. For this example, the reward is a *FlatReward* that just computes how many times step the *Agent* has sucessfully managed before breaking any rules. For more control on this reward, it is recommended to use look at the document of the Environment class.

More example of rewards are also available on the official document or [here](https://grid2op.readthedocs.io/en/latest/reward.html).

In [11]:
done = False
time_step = int(0)
cum_reward = 0.
obs = env.reset()
reward = env.reward_range[0]
while not done:
    act = my_agent.act(obs, reward, done) # chose an action to do, in this case "do nothing"
    obs, reward, done, info = env.step(act) # implement this action on the powergrid
    cum_reward += reward
    time_step += 1
    if time_step > max_iter:
        break

We can now evaluate how well this *agent* is performing:

In [12]:
print("This agent managed to survive {} timesteps".format(time_step))
print("It's final cumulated reward is {}".format(cum_reward))

This agent managed to survive 11 timesteps
It's final cumulated reward is 12239.267128824638


**NB** To the function "make" is highly customizable. For example, you can change the reward you are using to train your agent this way:

```python
from grid2op.Reward import L2RPNReward
env = grid2op.make_new(action_class=my_agent,
                       reward_class=L2RPNReward)
```

Because we thought using a single reward to train an agent in such a complex environment, we also gave the possibility to assess different rewards during training. This can be done with the following code:

```python

from grid2op.Reward import L2RPNReward, FlatReward
env = grid2op.make_new(action_class=my_agent,
                       reward_class=L2RPNReward,
                       other_rewards={"other_reward" : FlatReward })
```
These result of these reward can be accessed in the "info" return value of the call to env.step. See the official document of reward [here](https://grid2op.readthedocs.io/en/latest/reward.html) for more information.

## IV) More convenient ways to perform all these operations

All the above steps have been detailed as a "quick start", to give an example of the main classes of the Grid2Op package. Having to code all the above is quite tedious, but offers a lot of flexibility.

Implementing all this before starting to evaluate an agent can be quite tedious. What we expose here is a much shorter way to perfom all of the above. In this section we will expose 2 ways:
* The quickest way, using the grid2op.main API, most suited when basic computations need to be carried out.
* The recommended way using a *Runner*, it gives more flexibilities than the grid2op.main API but can be harder to configure.

For this section, we assume the same as before:
* The Agent is "Do Nothing"
* The Environment is the default Environment
* PandaPower is used as the backend
* The chronics comes from the files included in this package
* etc.

### IV.A) Using the grid2op.runner API

When only simple assessment need to be performed, the grid2op.main API is perfectly suited. This API can also be access with the command line:
```bash
python3 -m grid2op.main
```

We detail here its usage as an API, to assess the performance of a given Agent.

As opposed to building en environment from scratch (see the previous section) this requires much less effort: we don't need to initialize (instanciate) anything. All is carried out inside the Runner called by the *main* function.

We ask here 1 episode (eg. we play one scenario until: either the agent does a game over, or until the scenario ends). But this method would work as well if we more.

In [13]:
from grid2op.Runner import Runner
runner = Runner(**env.get_params_for_runner(), agentClass=DoNothingAgent)
res = runner.run(nb_episode=1, max_iter=max_iter)

A call of the single 2 lines above will:
* Create a valid environment
* Create a valid agent
* Assess how well an agent performs on one episode.

In [14]:
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at /home/donnotben/Documents/Grid2Op_dev/getting_started/grid2op/data/rte_case14_redisp/chronics/0
		 - cumulative reward: 10948.05
		 - number of time steps completed: 10 / 10


This is particularly suited to evaluate different agents, for example we can quickly evaluate a second agent. For the below example, we can import an agent class *PowerLineSwitch* whose job is to connect and disconnect the power lines in the power network. This *PowerLineSwitch* Agent will simulate the effect of disconnecting each powerline on the powergrid and take the best action found (its execution can take a long time, depending on the scenario and the amount of powerlines on the grid). **The execution of the code below can take a few moments**

In [15]:
from grid2op.Agent import PowerLineSwitch
runner = Runner(**env.get_params_for_runner(), agentClass=PowerLineSwitch)
res = runner.run(nb_episode=1, max_iter=max_iter)
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at /home/donnotben/Documents/Grid2Op_dev/getting_started/grid2op/data/rte_case14_redisp/chronics/0
		 - cumulative reward: 10950.26
		 - number of time steps completed: 10 / 10


Using this API it's also possible to store the results for a detailed examination of the aciton taken by the Agent. Note that writing on the hard drive has an overhead on the computation time.

To do this, only a simple argument need to be added to the *main* function call. An example can be found below (where the outcome of the experiment will be stored in the `saved_experiment_donothing` directory):

In [16]:
runner = Runner(**env.get_params_for_runner(),
                agentClass=PowerLineSwitch
               )
res = runner.run(nb_episode=1, max_iter=max_iter, path_save=os.path.abspath("saved_experiment_donothing"))
print("The results are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.2f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results are:
	For chronics located at /home/donnotben/Documents/Grid2Op_dev/getting_started/grid2op/data/rte_case14_redisp/chronics/0
		 - cumulative reward: 10950.26
		 - number of time steps completed: 10 / 10


In [17]:
!ls saved_experiment_donothing/1

actions.npy			  episode_meta.json   _parameters.json
agent_exec_times.npy		  episode_times.json  rewards.npy
disc_lines_cascading_failure.npy  observations.npy
env_modifications.npy		  other_rewards.json


All the informations saved are showed above. For more information about them, please don't hesitate to read the documentation of [Runner](https://grid2op.readthedocs.io/en/latest/runner.html).

**NB** A lot more of informations about *Action* is provided in the [2_Action_GridManipulation](2_Action_GridManipulation.ipynb) notebook. In the [3_TrainingAnAgent](3_TrainingAnAgent.ipynb) (last section) there is an quick example on how to read / write action from a saved repository.

In the notebook 7 more details are given as for the advantages of the runner, especially for post analysis of the agent performances.

The use of `make` + `runner` makes it easy to assess the performance of trained agent. Beside, Runner has been particularly integrated with other tools and makes easy the replay / post analysis of the episode. It is the recommended method to use in grid2op.

### IV.B) Using the "main" function

When only **really simple** assessment need to be performed, the grid2op.main API is perfectly suited. This API can also be access with the command line:
```bash
python3 -m grid2op.main
```

We detail here its usage as an API, to assess the performance of a given Agent.

As opposed to building en environment from scratch (see the previous section) this requires much less effort: we don't need to initialize (instanciate) anything. All is carried out inside the Runner called by the *main* function.

We ask here 1 episode (eg. we play one scenario until: either the agent does a game over, or until the scenario ends). But this method would work as well if we more.

In [18]:
from grid2op.main import main_run

res = main_run(nb_episode=1,
               agent_class=DoNothingAgent,
               path_casefile=powergrid_path,
               path_chronics=multi_episode_path,
               gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts, "max_iter": max_iter}
               )

The use of `make` + `runner` makes it easy to assess the performance of trained agent. Beside, Runner has been particularly integrated with other tools and makes easy the replay / post analysis of the episode. It is the recommended method to use in grid2op.